<a href="https://colab.research.google.com/github/MrAwesomeJr/comp-451-cat-bounding-box/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fiftyone

In [ ]:
import fiftyone as fo
import numpy as np

#LOADING DATA
dataset = fo.zoo.load_zoo_dataset(
              "open-images-v7",
              split="train",
              label_types=["detections"],
              classes=["Cat"],
            #   max_samples=10,
          )

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
import cv2

In [3]:
# batching is not necessary if we're using fasterrcnn_resnet50_fpn.
dataset = dataset[:500]
batch_size = 100
n_samples = len(dataset)
x_size = 267
y_size = 326

In [4]:
unbatched_data = []
unbatched_boxes = []

for sample in dataset:
    # Load and resize the image
    image = cv2.imread(sample['filepath'])
    resized_image = cv2.resize(image, (x_size, y_size))

    # Convert to tensor
    tensor_image = transforms.ToTensor()(resized_image)
    unbatched_data.append(tensor_image)

    # Extract bounding box for the 'Cat' label
    for detection in sample['ground_truth']['detections']:
        if detection['label'] == 'Cat':
            unbatched_boxes.append(detection['bounding_box'])
            break

targets = []

for i, box in enumerate(unbatched_boxes):
    #normalized coordinates to pixel values
    x_min, y_min, width, height = box
    x_max = x_min + width
    y_max = y_min + height

    #target dictionary
    target = {
        'boxes': torch.tensor([[x_min * x_size, y_min * y_size, x_max * x_size, y_max * y_size]], dtype=torch.float32),
        'labels': torch.tensor([17], dtype=torch.int64),  #17 is the label for 'Cat'
    }
    targets.append(target)

In [ ]:
print(f"Unbatched data length: {len(unbatched_data)}")
print(f"Tensor shape (example): {unbatched_data[0].shape}")

Unbatched data length: 500
Tensor shape (example): torch.Size([3, 326, 267])


In [ ]:
#TRAINING
import ssl
ssl._create_default_https_context=ssl._create_unverified_context
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=None, num_classes=2)
output=model(unbatched_data,targets)


In [ ]:
#MODEL EVALUATION

model.eval() # Set the model to evaluation mode

batch_size = 20  #processing in batches of 20 images at a time
predictions = []

for i in range(0, len(unbatched_data), batch_size):
    batch = unbatched_data[i:i + batch_size]
    print(i)
    # Run inference on the batch
    with torch.no_grad():
        predictions_batch = model(batch)

    predictions.extend(predictions_batch)
# Ensure inputs are lists
#with torch.no_grad():
   # predictions = model(unbatched_data)  # Only pass images for inference

# Print predictions
for i, prediction in enumerate(predictions):
    print(f"Image {i} Predictions:")
    print(prediction)

In [ ]:
#METRICS
from sklearn.metrics import average_precision_score
test_set = fo.zoo.load_zoo_dataset(
              "open-images-v7",
              split="test",
              label_types=["detections"],
              classes=["Cat"],
            #   max_samples=10,
          )
test_set = test_set[:500]
acc_score = accuracy_score(test_set, predictions)
print(f"Accuracy Score: {acc_score}")
avg_prec_score = average_precision_score(test_set, predictions)
print(f"Average Precision Score: {avg_prec_score}")

